**ETL**

In [42]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import seaborn as sns

In [43]:
import csv

In [44]:
data = pd.read_csv("movies_dataset.csv")

C:\Users\facun\AppData\Local\Temp\ipykernel_4508\133786677.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("movies_dataset.csv")


**PRIMERA PARTE: DASANIDAR COLUMNAS**

Comenzamos con la columna `belongs_to_collection` (Diccionario), en cada registro en caso de no ser nulo, se reemplazará solo con el nombre

In [45]:
for i in range(len(data)):
    try:
        if isinstance(eval(data["belongs_to_collection"][i]), dict):
            data["belongs_to_collection"][i] = eval(data["belongs_to_collection"][i])["name"]
    except:
        pass

C:\Users\facun\AppData\Local\Temp\ipykernel_4508\2878617107.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["belongs_to_collection"][i] = eval(data["belongs_to_collection"][i])["name"]


Para `genres` se añadirán 3 columnas más al DataFrame que serán `g1`, `g2`, `g3` y se agregaran los generos a cada columna respectivamente

In [46]:
pd.json_normalize(eval(data["genres"][2]))["name"][0]

'Romance'

Se desanida la columna genres (Diccionario)

In [47]:
data['g1'] = np.nan
data['g2'] = np.nan
data['g3'] = np.nan


for i in range(len(data)):
    try:
      df = pd.json_normalize(eval(data['genres'][i]))

      for j in range(len(df)):
        data[f'g{j+1}'][i] = df['name'][j]
    except:
        pass

C:\Users\facun\AppData\Local\Temp\ipykernel_4508\2090772770.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[f'g{j+1}'][i] = df['name'][j]


Desnidamos la columna data `production_companies` (Lista)

In [48]:
data['produc_com1'] = np.nan
data['produc_com2'] = np.nan

for i in range(len(data)):
    try:
        df = pd.json_normalize(eval(data['production_companies'][i]))

        for j in range(len(df)):
            data[f'produc_com{j+1}'][i] = df['name'][j]
    except:
        continue

C:\Users\facun\AppData\Local\Temp\ipykernel_4508\3212550824.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[f'produc_com{j+1}'][i] = df['name'][j]
C:\Users\facun\AppData\Local\Temp\ipykernel_4508\3212550824.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[f'produc_com{j+1}'][i] = df['name'][j]


Se desanidará la columna `production_countries` (Lista)

In [49]:
data['production_countries'][34556]

"[{'iso_3166_1': 'IT', 'name': 'Italy'}]"

In [50]:
data['country'] = np.nan

for i in range(len(data)):
    try:
        df = pd.json_normalize(eval(data['production_countries'][i]))

        for j in range(len(df)):
            data['country'][i] = df['name'][j]
    except:
        continue

C:\Users\facun\AppData\Local\Temp\ipykernel_4508\2701637996.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['country'][i] = df['name'][j]


Se desanidará la columna `spoken_languages`

In [51]:
data['language'] = np.nan

for i in range(len(data)):
    try:
        df = pd.json_normalize(eval(data['spoken_languages'][i]))

        for j in range(len(df)):
            data['language'][i] = df['name'][j]
    except:
        continue

C:\Users\facun\AppData\Local\Temp\ipykernel_4508\640928165.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['language'][i] = df['name'][j]


In [52]:
columnas = ['genres', 'production_companies',
       'production_countries','spoken_languages']

data = data.drop(columns=columnas)

In [53]:
data

,adult,belongs_to_collection,budget,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,release_date,revenue,runtime,status,tagline,title,video,vote_average,vote_count,g1,g2,g3,produc_com1,produc_com2,country,language
0,False,Toy Story Collection,30000000,http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,1995-10-30,373554033.0,81.0,Released,NaN,Toy Story,False,7.7,5415.0,Animation,Comedy,Family,Pixar Animation Studios,NaN,United States of America,English
1,False,NaN,65000000,NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,1995-12-15,262797249.0,104.0,Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,Adventure,Fantasy,Family,TriStar Pictures,Teitler Film,United States of America,Français
2,False,Grumpy Old Men Collection,0,NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,1995-12-22,0.0,101.0,Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,Romance,Comedy,NaN,Warner Bros.,Lancaster Gate,United States of America,English
3,False,NaN,16000000,NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.859495,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,1995-12-22,81452156.0,127.0,Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0,Comedy,Drama,Romance,Twentieth Century Fox Film Corporation,NaN,United States of America,English
4,False,Father of the Bride Collection,0,NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.387519,/e64sOI48hQXyru7naBFyssKFxVd.jpg,1995-02-10,76578911.0,106.0,Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0,Comedy,NaN,NaN,Sandollar Productions,Touchstone Pictures,United States of America,English
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45461,False,NaN,0,http://www.imdb.com/title/tt6209470/,439050,tt6209470,fa,رگ خواب,Rising and falling between a man and woman.,0.072051,/jldsYflnId4tTWPx8es3uzsB1I8.jpg,NaN,0.0,90.0,Released,Rising and falling between a man and woman,Subdue,False,4.0,1.0,Drama,Family,NaN,NaN,NaN,Iran,فارسی
45462,False,NaN,0,NaN,111109,tt2028550,tl,Siglo ng Pagluluwal,An artist struggles to finish his work while a...,0.178241,/xZkmxsNmYXJbKVsTRLLx3pqGHx7.jpg,2011-11-17,0.0,360.0,Released,NaN,Century of Birthing,False,9.0,3.0,Drama,NaN,NaN,Sine Olivia,NaN,Philippines,
45463,False,NaN,0,NaN,67758,tt0303758,en,Betrayal,"When one of her hits goes wrong, a professiona...",0.903007,/d5bX92nDsISNhu3ZT69uHwmfCGw.jpg,2003-08-01,0.0,90.0,Released,A deadly game of wits.,Betrayal,False,3.8,6.0,Action,Drama,Thriller,American World Pictures,NaN,United States of America,English
45464,False,NaN,0,NaN,227506,tt0008536,en,Satana likuyushchiy,"In a small town live two brothers, one a minis...",0.003503,/aorBPO7ak8e8iJKT5OcqYxU3jlK.jpg,1917-10-21,0.0,87.0,Released,NaN,Satan Triumphant,False,0.0,0.0,NaN,NaN,NaN,Yermoliev,NaN,Russia,NaN


Los valores nulos de los campos `revenue`, `budget` deben ser rellenados por el número 0

In [54]:
data['budget'] = data['budget'].fillna(0)
data['revenue'] = data['revenue'].fillna(0)

Los valores nulos del campo `release date` deben ser eliminados

In [55]:
nombre_columna = 'release_date'
data = data.dropna(subset=[nombre_columna])

 De haber fechas, deberán tener el formato `AAAA-mm-dd`, además deberán crear la columna `release_year` donde extraerán el año de la fecha de estreno.


In [56]:
data['release_date'] =  pd.to_datetime(data['release_date'], errors='coerce')

C:\Users\facun\AppData\Local\Temp\ipykernel_4508\4179229598.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['release_date'] =  pd.to_datetime(data['release_date'], errors='coerce')


In [57]:
data['release_year'] = data['release_date'].dt.strftime('%Y')

C:\Users\facun\AppData\Local\Temp\ipykernel_4508\256904507.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['release_year'] = data['release_date'].dt.strftime('%Y')


Acá transformamos las columnas `release_date` y `release_year` de formato Object a Date


In [58]:
data['release_year'] = pd.to_datetime(data['release_date'], format='%Y')

C:\Users\facun\AppData\Local\Temp\ipykernel_4508\1601711881.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['release_year'] = pd.to_datetime(data['release_date'], format='%Y')


In [59]:
data['release_year'] = data['release_year'].dt.to_period('Y')

C:\Users\facun\AppData\Local\Temp\ipykernel_4508\3553755221.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['release_year'] = data['release_year'].dt.to_period('Y')


Crear la columna con el retorno de inversión, llamada `return` con los campos `revenue` y `budget`, dividiendo estas dos últimas `revenue/budget`, cuando no hay datos disponibles para calcularlo, deberá tomar el valor `0`.


Acá se noto unos datos extraños por lo que se decidió reemplazarlos por cero para poder hacer el return correctamente

In [60]:
data['budget'] = data['budget'].replace('/zaSf5OG7V8X8gqFvly88zDdRm46.jpg', 0)
data['budget'] = data['budget'].replace('/zV8bHuSL6WXoD6FWogP9j4x80bL.jpg', 0)
data['budget'] = data['budget'].replace('/ff9qCepilowshEtG2GYWwzt2bs4.jpg', 0)
data['budget'] = data['budget'].astype(int)

C:\Users\facun\AppData\Local\Temp\ipykernel_4508\118796259.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['budget'] = data['budget'].replace('/zaSf5OG7V8X8gqFvly88zDdRm46.jpg', 0)
C:\Users\facun\AppData\Local\Temp\ipykernel_4508\118796259.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['budget'] = data['budget'].replace('/zV8bHuSL6WXoD6FWogP9j4x80bL.jpg', 0)
C:\Users\facun\AppData\Local\Temp\ipykernel_4508\118796259.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a

In [61]:
def calcular_columna(row):
    if row['budget'] != 0 and row['revenue'] != 0:
        return row['revenue'] / row['budget']
    else:
        return np.nan
data['return'] = data.apply(calcular_columna, axis=1)

C:\Users\facun\AppData\Local\Temp\ipykernel_4508\3988146083.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['return'] = data.apply(calcular_columna, axis=1)


Eliminar las columnas que no serán utilizadas: `video`,`imdb_id`,`adult`,`original_title`,`poster_path` y `homepage`.


In [62]:
columnas = ['video', 'imdb_id', 'adult', 'original_title',
            'poster_path', 'homepage']

data = data.drop(columns=columnas)

In [63]:
data

,belongs_to_collection,budget,id,original_language,overview,popularity,release_date,revenue,runtime,status,tagline,title,vote_average,vote_count,g1,g2,g3,produc_com1,produc_com2,country,language,release_year,return
0,Toy Story Collection,30000000,862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,1995-10-30,373554033.0,81.0,Released,NaN,Toy Story,7.7,5415.0,Animation,Comedy,Family,Pixar Animation Studios,NaN,United States of America,English,1995,12.451801
1,NaN,65000000,8844,en,When siblings Judy and Peter discover an encha...,17.015539,1995-12-15,262797249.0,104.0,Released,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0,Adventure,Fantasy,Family,TriStar Pictures,Teitler Film,United States of America,Français,1995,4.043035
2,Grumpy Old Men Collection,0,15602,en,A family wedding reignites the ancient feud be...,11.7129,1995-12-22,0.0,101.0,Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,6.5,92.0,Romance,Comedy,NaN,Warner Bros.,Lancaster Gate,United States of America,English,1995,NaN
3,NaN,16000000,31357,en,"Cheated on, mistreated and stepped on, the wom...",3.859495,1995-12-22,81452156.0,127.0,Released,Friends are the people who let you be yourself...,Waiting to Exhale,6.1,34.0,Comedy,Drama,Romance,Twentieth Century Fox Film Corporation,NaN,United States of America,English,1995,5.090760
4,Father of the Bride Collection,0,11862,en,Just when George Banks has recovered from his ...,8.387519,1995-02-10,76578911.0,106.0,Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,5.7,173.0,Comedy,NaN,NaN,Sandollar Productions,Touchstone Pictures,United States of America,English,1995,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45460,NaN,0,30840,en,"Yet another version of the classic epic, with ...",5.683753,1991-05-13,0.0,104.0,Released,NaN,Robin Hood,5.7,26.0,Drama,Action,Romance,Westdeutscher Rundfunk (WDR),Working Title Films,United States of America,English,1991,NaN
45462,NaN,0,111109,tl,An artist struggles to finish his work while a...,0.178241,2011-11-17,0.0,360.0,Released,NaN,Century of Birthing,9.0,3.0,Drama,NaN,NaN,Sine Olivia,NaN,Philippines,,2011,NaN
45463,NaN,0,67758,en,"When one of her hits goes wrong, a professiona...",0.903007,2003-08-01,0.0,90.0,Released,A deadly game of wits.,Betrayal,3.8,6.0,Action,Drama,Thriller,American World Pictures,NaN,United States of America,English,2003,NaN
45464,NaN,0,227506,en,"In a small town live two brothers, one a minis...",0.003503,1917-10-21,0.0,87.0,Released,NaN,Satan Triumphant,0.0,0.0,NaN,NaN,NaN,Yermoliev,NaN,Russia,NaN,1917,NaN


In [64]:
df = pd.read_csv("credits.csv")

In [65]:
df['crew'] = df['crew'].apply(eval)

In [66]:
list_crew = []

In [67]:
for i, j in enumerate(df['crew']):
    dfcrew = pd.json_normalize(df['crew'][i])
    list_crew.append(dfcrew)


In [68]:
df_crew = pd.concat(list_crew, ignore_index=True)

In [69]:
df_crew

,credit_id,department,gender,id,job,name,profile_path
0,52fe4284c3a36847f8024f49,Directing,2,7879,Director,John Lasseter,/7EdqiNbr4FRjIhKHyPPdFfEEEFG.jpg
1,52fe4284c3a36847f8024f4f,Writing,2,12891,Screenplay,Joss Whedon,/dTiVsuaTVTeGmvkhcyJvKp2A5kr.jpg
2,52fe4284c3a36847f8024f55,Writing,2,7,Screenplay,Andrew Stanton,/pvQWsu0qc8JFQhMVJkTHuexUAa1.jpg
3,52fe4284c3a36847f8024f5b,Writing,2,12892,Screenplay,Joel Cohen,/dAubAiZcvKFbboWlj7oXOkZnTSu.jpg
4,52fe4284c3a36847f8024f61,Writing,0,12893,Screenplay,Alec Sokolow,/v79vlRYi94BZUQnkkyznbGUZLjT.jpg
...,...,...,...,...,...,...,...
464309,52fe4776c3a368484e0c8399,Sound,0,549356,Original Music Composer,Richard McHugh,None
464310,52fe4776c3a368484e0c839f,Camera,2,58818,Director of Photography,João Fernandes,None
464311,533bccebc3a36844cf0011a7,Directing,0,1085341,Director,Yakov Protazanov,/yyjbGdCs2ZN6IlZNCfmBWyuRDlt.jpg
464312,58ebbc26925141281908aa0a,Production,2,1195656,Producer,Joseph N. Ermolieff,None


In [70]:
data_list = []
for i in range(len(df)):
    data = pd.DataFrame(df.crew[i])
    data['id'] = df.id[i]
    data_list.append(data)


In [71]:
dfcast = pd.concat(data_list, ignore_index=True)
prueba = dfcast[dfcast['job'] =='Director']

In [72]:
final = prueba.drop(columns=['profile_path', 'credit_id', 'gender', 'department', 'job'])

In [73]:
final

,id,name
0,862,John Lasseter
109,8844,Joe Johnston
122,15602,Howard Deutch
126,31357,Forest Whitaker
141,11862,Charles Shyer
...,...,...
464291,439050,Hamid Nematollah
464300,111109,Lav Diaz
464306,67758,Mark L. Lester
464311,227506,Yakov Protazanov


In [74]:
dfdirectores = pd.merge(data, final, on='id')

In [75]:
dfdirectores

,credit_id,department,gender,id,job,name_x,profile_path,name_y
0,593e676c92514105b702e68e,Directing,0,461257,Director,Daisy Asquith,None,Daisy Asquith


In [76]:
pd.options.display.max_columns = 100